In [4]:
#Import Dependencies

import os
import numpy as np
import time
import cv2
from PIL import Image

import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.models import Sequential
from keras.utils import np_utils
from matplotlib import pyplot as plt

In [14]:
#
#path = os.getcwd()
path = os.path.join(os.getcwd(),'skin-cancer-malignant-vs-benign','train')
print(path)

/Users/abhinavmathur/ML Repository/Sample-Codes/Computer Vision/skin-cancer-malignant-vs-benign/train


In [16]:
types = os.listdir(path)
types.pop(0)
types

['malignant', 'benign']

In [17]:
def Reverse(lst): 
    return [ele for ele in reversed(lst)] 

read = Reverse(types)
read

['benign', 'malignant']

In [71]:
# Define label and data lists

start = time.time()
Data = []
res = []

for typ in read:
    outcome = read.index(typ)
    #print(outcome)
    file_dir = os.path.join(path,typ)
    files = os.listdir(file_dir)
    for file in files:
        img = cv2.imread(os.path.join(path,typ,file))
        Img = Image.fromarray(img, 'RGB')
        Img2 = Img.resize((150,150))
        Data.append(np.array(Img2))
        res.append(outcome)
        
end = time.time()
print(end-start," seconds")

27.408655881881714  seconds


In [29]:
Images = np.array(Data)
res2 = np.array(res)
Images.shape

(2637, 150, 150, 3)

2637 Images of resolution 150 X 150 for RGB

In [31]:
# Reshufle Images and 

indexes=np.arange(Images.shape[0]) #Get the total count of rows in an array
np.random.shuffle(indexes) # Randomly shuffle row numbers
images=Images[indexes] # Rearrange rows in the images data
infections=res2[indexes] # Rearrange outcomes

In [36]:
# Split the data into train and test datasets by taking a 80:20 split

(X_train, X_test) = images[(int)(0.3*len(images)):],images[:(int)(0.3*len(images))]
(Y_train, Y_test) = infections[(int)(0.3*len(infections)):],infections[:(int)(0.3*len(infections))]
print(len(X_train),len(X_test),len(images))

1846 791 2637


In [37]:
print(len(Y_train),len(Y_test))

1846 791


In [38]:
# Normalize the data by dividing with 255 pixels
X_train = X_train.astype('float32')/255 # As we are working on image data we are normalizing data by divinding 255.
X_test = X_test.astype('float32')/255

In [39]:
# As this is a classification problem we will need to complete a one hot encoding for all categorical variables

Y_train = keras.utils.to_categorical(Y_train,len(np.unique(infections)))
Y_test = keras.utils.to_categorical(Y_test,len(np.unique(infections)))

In [44]:
# Defining and training a CNN

model=Sequential()
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu",input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="sigmoid"))
model.add(Dropout(0.2))
model.add(Dense(2,activation="sigmoid"))#2 represent output layer neurons 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 150, 150, 32)      416       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 75, 75, 64)        8256      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 37, 37, 128)       32896     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 18, 18, 128)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 18, 18, 128)       0         
__________

In [45]:
# compile the model with loss as categorical_crossentropy and using adam optimizer you can test result by trying RMSProp as well as Momentum
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Fit the model with min batch size as 50[can tune batch size to some factor of 2^power ] 
model.fit(X_train,Y_train,batch_size=70,epochs=50,verbose=1)

Epoch 1/50
1846/1846 [==============================] - 73s 39ms/step - loss: 0.8481 - acc: 0.5699
Epoch 2/50
1846/1846 [==============================] - 70s 38ms/step - loss: 0.5168 - acc: 0.7356
Epoch 3/50
1846/1846 [==============================] - 83s 45ms/step - loss: 0.4610 - acc: 0.7649
Epoch 4/50
1846/1846 [==============================] - 75s 41ms/step - loss: 0.4897 - acc: 0.7516
Epoch 5/50
1846/1846 [==============================] - 82s 44ms/step - loss: 0.4173 - acc: 0.7925
Epoch 6/50
1846/1846 [==============================] - 86s 47ms/step - loss: 0.4337 - acc: 0.7849
Epoch 7/50
1846/1846 [==============================] - 81s 44ms/step - loss: 0.3919 - acc: 0.8047
Epoch 8/50
1846/1846 [==============================] - 101s 55ms/step - loss: 0.3750 - acc: 0.8164
Epoch 9/50
1846/1846 [==============================] - 74s 40ms/step - loss: 0.3813 - acc: 0.8093
Epoch 10/50
1846/1846 [==============================] - 74s 40ms/step - loss: 0.3361 - acc: 0.8334
Epoch 11

In [46]:
accuracy = model.evaluate(X_test, Y_test, verbose=1)

print('\n', 'Test_Accuracy:-', accuracy[1])

791/791 [==============================] - 9s 11ms/step

 Test_Accuracy:- 0.8565107453638021


In [48]:
# Read test data as done with training data

test_path = os.path.join(os.getcwd(),'skin-cancer-malignant-vs-benign','test')
types = os.listdir(test_path)
types

['malignant', 'benign']

In [49]:
read_test = Reverse(types)
read_test

['benign', 'malignant']

In [60]:
# Define label and data lists

start = time.time()
Data_test = []
res_test = []

for typ in read_test:
    outcome = read_test.index(typ)
    #print(outcome)
    file_dir = os.path.join(test_path,typ)
    files = os.listdir(file_dir)
    for file in files:
        img = cv2.imread(os.path.join(test_path,typ,file))
        Img = Image.fromarray(img, 'RGB')
        Img2 = Img.resize((150,150))
        Data_test.append(np.array(Img2))
        res_test.append(outcome)
        
end = time.time()
print(end-start," seconds")

1.796050786972046  seconds


In [61]:
Images = np.array(Data_test)
res2 = np.array(res_test)
Images.shape

(660, 150, 150, 3)

In [62]:
len(res2)

660

In [63]:
Images2 = Images.astype('float32')/255

In [64]:
Y = keras.utils.to_categorical(res2,len(np.unique(res2)))
Y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [65]:
read_test

['benign', 'malignant']

In [70]:
accuracy = model.evaluate(Images2, Y, verbose=1)

print('\n', 'Test_Accuracy:-', accuracy[1])

660/660 [==============================] - 7s 11ms/step

 Test_Accuracy:- 0.8378787886012684


In [72]:
# Saving the keras model

model.save('Malignant Skin Tumour Detection CNN Keras')